## Run inference on a sample of images using all models to visually compare segmentation

In [1]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torchvision.transforms as T
import segmentation_models_pytorch as smp

In [ ]:
#Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_paths = [
    "../best_deeplabmodels/V1_only_cmp_.pth",
    "../best_deeplabmodels/V2_only_mapillary_.pth",
    "../best_deeplabmodels/V3_combined_.pth"
    "../best_deeplabmodels/V4_cmp_ft_decoder_.pth",
    "../best_deeplabmodels/V5_cmp_ft_encoder_.pth",
    "../best_deeplabmodels/V6_cmp_ft_encoder_l34.pth"
]

image_dir = "../panos/cropped_panos"
output_dir = "../inference/compare_models"
os.makedirs(output_dir, exist_ok=True)

num_classes = 4
ignore_class = 0
image_size = (512, 1024)

#Transform
transform = T.Compose([
    T.Resize(image_size),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

#Colors for masks
colors = {
    0: (0, 0, 0, 0),          # transparent- class 0
    1: (0, 0, 255, 100),      # background- class 1
    2: (0, 255, 0, 100),      # porosity- class 2
    3: (255, 0, 0, 100)       # wall- class 3
}

#Select 50 random images
all_filenames = sorted(os.listdir(image_dir))
random.seed(42)  # para reproducibilidad
sample_filenames = random.sample(all_filenames, 50)

#Load each model
def load_model(model_path):
    model = smp.DeepLabV3Plus(
        encoder_name="resnet101",
        encoder_weights="imagenet",
        in_channels=3,
        classes=num_classes
    )
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

#Process all models
for model_path in model_paths:
    model_name = os.path.splitext(os.path.basename(model_path))[0]
    print(f"Processing model: {model_name}")

    #Create directory for this model
    model_output_dir = os.path.join(output_dir, model_name)
    os.makedirs(model_output_dir, exist_ok=True)

    #Load model
    model = load_model(model_path)

    #Results for CSV
    results = []

    #Process sample images
    for filename in sample_filenames:
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path).convert("RGB")
        input_tensor = transform(img).unsqueeze(0).to(device)

        #Run inference
        with torch.no_grad():
            output = model(input_tensor)
            pred = torch.argmax(output.squeeze(), dim=0).cpu().numpy()

        #Extract IDs
        base = os.path.splitext(filename)[0]
        edge_id, pano_id = base.split("_", 1)

        #Calculate percentages per class
        total_pixels = (pred != ignore_class).sum()
        percentages = []
        for cls in range(1, num_classes):
            class_pixels = (pred == cls).sum()
            pct = (class_pixels / total_pixels * 100) if total_pixels > 0 else 0.0
            percentages.append(round(pct, 4))

        #Create masks
        pred_resized = Image.fromarray(pred.astype(np.uint8)).resize(img.size, resample=Image.NEAREST)
        pred_np = np.array(pred_resized)
        mask_rgba = Image.new("RGBA", img.size, (0, 0, 0, 0))
        mask_pixels = mask_rgba.load()
        for y in range(pred_np.shape[0]):
            for x in range(pred_np.shape[1]):
                cls = pred_np[y, x]
                mask_pixels[x, y] = colors.get(cls, (0, 0, 0, 0))

        #Overlay masks and save
        img_rgba = img.convert("RGBA")
        overlayed = Image.alpha_composite(img_rgba, mask_rgba)
        overlayed.save(os.path.join(model_output_dir, f"{base}_overlay.png"))

        #Save results
        results.append({
            "edge_id": edge_id,
            "pano_id": pano_id,
            "pct_background": percentages[0],  # Class 1
            "pct_porosity": percentages[1],   # Class 2
            "pct_wall": percentages[2],       # Class 3
        })

    #Save CSV for this model
    results_df = pd.DataFrame(results)
    results_csv_path = os.path.join(model_output_dir, f"{model_name}_results.csv")
    results_df.to_csv(results_csv_path, index=False)
    print(f"Results saved to {results_csv_path}")


Processing model: V4_cmp_ft_decoder_
Resultados guardados en ../inference/compare_models\V4_cmp_ft_decoder_\V4_cmp_ft_decoder__results.csv
Processing model: V5_cmp_ft_encoder_
Resultados guardados en ../inference/compare_models\V5_cmp_ft_encoder_\V5_cmp_ft_encoder__results.csv
Processing model: V6_cmp_ft_encoder_l34
Resultados guardados en ../inference/compare_models\V6_cmp_ft_encoder_l34\V6_cmp_ft_encoder_l34_results.csv
